In [1]:
# syft absolute
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    # node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=8316910b45cb42a7a5967c96eb1c00a6 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")

Logged into <test_l: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: High side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.20it/s]

Uploading: numpy-data


# Make Requests

In [7]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


# compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [8]:
res = client_low_ds.code.request_code_execution(compute_mean)

In [9]:
res

syft.service.request.request.Request

In [10]:
node_low.python_node.get_service("usercodeservice").stash.partition.data

{<UID: 8484c4fc94c84dae84fa023abdac7c7d>: syft.service.code.user_code.UserCode}

In [11]:
client_low_ds.code[0].status

{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}

In [12]:
# client_low.requests[0].accept_by_depositing_result(10.0)
client_low.requests[0].approve()

Approving request for domain test_l


SyftSuccess: Request 51e3f385a50d4acba2f9366d67d888d1 changes applied

In [13]:
data = client_low_ds.datasets[0].assets[0]

res = client_low_ds.code.compute_mean(data=data, blocking=False)

res

```python
class Job:
    id: UID = 1c97c22c64b94a6bb7f87f416c773d7e
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [ ]:
res.wait()

In [16]:
client_low_ds.code.compute_mean(data=data).get()

override False
output_history [syft.service.output.output_service.ExecutionOutput] ab3580afd6a34bd4abc8d6cafed36243
output_history [syft.service.output.output_service.ExecutionOutput] ab3580afd6a34bd4abc8d6cafed36243


7.0

In [17]:
client_low_ds.code[0].output_policy

```python
class OutputPolicyExecuteOnce:
  id: str = ab3580afd6a34bd4abc8d6cafed36243

```

In [18]:
node_low.python_node.get_service("outputservice").stash.partition.data

{<UID: 6042ce1735cc48848861ff8796ef33f7>: syft.service.output.output_service.ExecutionOutput}

In [19]:
node_low.python_node.get_service("outputservice").stash.partition.permissions

{<UID: 6042ce1735cc48848861ff8796ef33f7>: {'5e11b6e4d634265827ebb4bb330e8e15996eb82027e0661f431cca7e926174ca_WRITE', '5e11b6e4d634265827ebb4bb330e8e15996eb82027e0661f431cca7e926174ca_READ', '5e11b6e4d634265827ebb4bb330e8e15996eb82027e0661f431cca7e926174ca_EXECUTE', '5e11b6e4d634265827ebb4bb330e8e15996eb82027e0661f431cca7e926174ca_OWNER'}}

In [20]:
client_low_ds.verify_key

5e11b6e4d634265827ebb4bb330e8e15996eb82027e0661f431cca7e926174ca

In [21]:
uid = client_low_ds.code[0].output_history[0].output_policy_id
client_low_ds.output.get_by_output_policy_id(uid)

[syft.service.output.output_service.ExecutionOutput]

In [22]:
client_low.sync.get_state()

syft.service.sync.sync_state.SyncState